In [2]:
import numpy as np
import math 
import matplotlib.pyplot as plt
import pandas as pd
df=pd.read_csv("./data/steam.csv")

Problématique : Le prix d'un jeu vidéo a-t-il une influence significative et systématique sur les ventes de ce dernier ?

In [3]:
##Partie 1 :
